In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import KFold, train_test_split
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# This may raise an exception in earlier versions of Jupyter
%config InlineBackend.figure_format = 'retina'

In [2]:
data_df = pd.read_csv('game_data/preprocessing_data.csv')
data_df.head()

,CUST_SEX,CUST_AGE,BUYERPART,CAR_AGE,CAR_PRICE,IS_LOAN,LOAN_PERIED,LOAN_AMOUNT,F_INSORNOT,ALL_BUYINS_N,...,CAR_MODEL_车型13,CAR_MODEL_车型2,CAR_MODEL_车型3,CAR_MODEL_车型4,CAR_MODEL_车型5,CAR_MODEL_车型6,CAR_MODEL_车型7,CAR_MODEL_车型8,CAR_MODEL_车型9,CAR_MODEL_nan
0,2,28.0,1,23.558438,360.277671,1,5.0,254.950976,1.0,1.0,...,0,1,0,0,0,0,0,0,0,0
1,1,27.0,1,23.366643,360.277671,1,4.0,264.499527,1.0,2.0,...,0,1,0,0,0,0,0,0,0,0
2,1,43.0,1,22.383029,360.277671,1,5.0,276.224546,1.0,1.0,...,0,1,0,0,0,0,0,0,0,0
3,1,42.0,1,23.323808,346.121366,1,4.0,282.241032,1.0,2.0,...,0,1,0,0,0,0,0,0,0,0
4,2,29.0,1,22.934690,360.277671,1,4.0,223.606798,1.0,2.0,...,0,1,0,0,0,0,0,0,0,0


In [3]:
result = data_df
x = result.drop('IS_LOST',axis=1)
y = result['IS_LOST']

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state = 0)

In [5]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

## XGBoost

In [8]:
xg_train = xgb.DMatrix(x_train, label=y_train)
xg_test = xgb.DMatrix(x_test, label=y_test)

In [10]:
xgb_params = {
    'objective': 'binary:logistic',
}
watchlist = [(xg_train, 'train'), (xg_test, 'test')]

In [16]:
%%time
##训练部分
clf = xgb.train(xgb_params, xg_train, 30, watchlist );

[0]	train-error:0.22002	test-error:0.21495
[1]	train-error:0.21827	test-error:0.21370
[2]	train-error:0.21782	test-error:0.21377
[3]	train-error:0.21810	test-error:0.21265
[4]	train-error:0.21807	test-error:0.21265
[5]	train-error:0.21734	test-error:0.21238
[6]	train-error:0.21660	test-error:0.21238
[7]	train-error:0.21643	test-error:0.21370
[8]	train-error:0.21581	test-error:0.21423
[9]	train-error:0.21558	test-error:0.21370
[10]	train-error:0.21474	test-error:0.21456
[11]	train-error:0.21443	test-error:0.21383
[12]	train-error:0.21437	test-error:0.21370
[13]	train-error:0.21412	test-error:0.21344
[14]	train-error:0.21383	test-error:0.21344
[15]	train-error:0.21355	test-error:0.21410
[16]	train-error:0.21299	test-error:0.21416
[17]	train-error:0.21237	test-error:0.21397
[18]	train-error:0.21245	test-error:0.21383
[19]	train-error:0.21197	test-error:0.21351
[20]	train-error:0.21115	test-error:0.21351
[21]	train-error:0.21073	test-error:0.21410
[22]	train-error:0.21059	test-error:0.2140

In [27]:
clf.get_fscore()

{'CAR_AGE': 243,
 'ONCE_PAY': 151,
 'ALL_BUYINS_N': 91,
 'LOAN_AMOUNT': 143,
 'CAR_PRICE': 108,
 'IS_LOAN': 16,
 'CUST_AGE': 135,
 'LOAN_PERIED': 27,
 'CAR_MODEL_车型3': 17,
 'F_INSORNOT': 14,
 'CAR_MODEL_车型9': 10,
 'GLASSBUYSEPARATE_CNT': 26,
 'CAR_MODEL_车型6': 5,
 'CAR_MODEL_车型7': 5,
 'CUST_SEX': 13,
 'SII_CNT': 14,
 'DLRSI_CNT': 18,
 'CAR_MODEL_车型4': 2,
 'CAR_MODEL_车型5': 6,
 'BUYERPART': 6,
 'CAR_MODEL_车型2': 6,
 'CAR_MODEL_车型13': 2,
 'CAR_MODEL_车型11': 2}